In [26]:
import cv2
import os

In [27]:
cascPathface = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
cascPatheyes = os.path.dirname(cv2.__file__) + "/data/haarcascade_eye_tree_eyeglasses.xml"

faceCascade = cv2.CascadeClassifier(cascPathface)
eyeCascade = cv2.CascadeClassifier(cascPatheyes)

mask_image = cv2.imread('spiderman.png')

In [ ]:
video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        faceROI = frame[y:y+h, x:x+w]
        eyes = eyeCascade.detectMultiScale(faceROI)
        
        for (x2, y2, w2, h2) in eyes:
            eye_center = (x + x2 + w2 // 2, y + y2 + h2 // 2)
            radius = int(round((w2 + h2) * 0.25))
            frame = cv2.circle(frame, eye_center, radius, (255, 0, 0), 4)

        # Calculate scaling factor and resize the mask image
        scale_factor = max(w / mask_image.shape[1], h / mask_image.shape[0])
        mask_image_resized = cv2.resize(mask_image, None, fx=scale_factor, fy=scale_factor)
        mask_h, mask_w, _ = mask_image_resized.shape
        
        # Adjust the mask position to align with the face region
        mask_x1 = max(x - int((mask_w - w) / 2), 0)
        mask_x2 = min(mask_x1 + mask_w, frame.shape[1])
        mask_y1 = max(y - int((mask_h - h) / 2), 0)
        mask_y2 = min(mask_y1 + mask_h, frame.shape[0])

        # Mask region within frame boundaries
        mask_image_resized = mask_image_resized[(mask_y2 - mask_y1) - h:y + (mask_y2 - mask_y1), 
                                                (mask_x2 - mask_x1) - w:x + (mask_x2 - mask_x1)]
        
        # Apply the resized mask image
        frame[y:y+h, x:x+w] = cv2.addWeighted(frame[y:y+h, x:x+w], 1, mask_image_resized, 1, 0)
        
        # Display the resulting frame
        cv2.imshow("Filter Window", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
video_capture.release()
cv2.destroyAllWindows()